### 👀교차 검증(Cross Validation)
- 기존 방식에서는 데이터 세트에서 학습 데이터 세트와 테스트 데이터 세트를 분리한 뒤 모델 검증을 진행한다.
- 교차 검증 시, 학습 데이터를 다시 분할하여 학습 데이터와 모델 성능을 1차 평가하는 검증 데이터로 나눈다.
<img src="./images/cross_validation01.png" width="500" style="margin-left: -30px">

### 교차 검증의 장단점
- 👍특정 데이터 세트에 대한 과적합 방지
- 👍데이터 세트 규모가 적을 시 과소적합 방지
- 👎모델 훈련, 모델 평가에 소요되는 시간 증가  
⛳ 과적합을 피하고 하이퍼 파라미터를 튜닝함으로써 모델을 일반화하고 신뢰성을 증가시키기 위해서 사용한다.

### 교차 검증의 종류
##### K-Fold
- k개의 데이터 폴드 세트를 만든 뒤 k번만큼 학습과 검증 평가를 반복하여 수행하는 방식.
- 학습 데이터와 검증 데이터를 정확히 자르기 때문에 타겟 데이터의 비중이 한 곳으로 치중될 수 있다.
- 예를 들어, 0, 1, 2, 중에서 0, 1, 두 가지만 잘라서 검증하게 되면 다른 하나의 타겟 데이터를 예측할 수 없게 된다.
- Stratified K-Fold로 해결한다.
##### Stratified K-Fold
- K-Fold와 마찬가지로 k번 수행하지만, 폴드 세트를 만들 때 학습 데이터 세트와 검증 데이터 세트가 가지는 타겟 분포도가 유사하도록 검증한다.
- 타겟 데이터의 비중을 항상 똑같게 자르기 때문에 데이터가 한 곳으로 치중되는 것을 방지한다.
<img src="./images/cross_validation02.png" width="500" style="margin-top:20px; margin-bottom:20px; margin-left: -30px">
##### GridSearchCV
- 교차 검증과 최적의 하이퍼 파라미터 튜닝을 한 번에 할 수 있는 객체이다.
- max_depth와 min_samples_split에 1차원 정수형 list를 전달하면, 2차원으로 결합하여 격자(Grid)를 만들고, 이 중 최적의 점을 찾아낸다.
- 딥러닝에서는 학습 속도가 머신러닝에 비해 느리고, 레이어(층)가 깊어질 수록 조정해주어야 할 하이퍼 파라미터 값이 많아지기 때문에, RandomSearchCV에서 대략적인 범위를 찾은 다음, GridSearchCV로 디테일을 조정하는 방식을 사용한다.
<img src="./images/grid_search_cv.png" width="500" style="margin-top: 20px; margin-left: -30px">

##### 붓꽃 데이터로 교차 검증

In [10]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
features, targets = iris.data, iris.target

target_df = pd.DataFrame(targets, columns=['target'])
target_df.value_counts()

target
0         50
1         50
2         50
Name: count, dtype: int64

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

decision_tree_classifier = DecisionTreeClassifier(min_samples_leaf=6, random_state=124)
kfold = KFold(n_splits=5)

##### KFold.split(feature)
- features만 전달하고 학습용, 검증용 행 번호를 array로 리턴한다.

In [12]:
for train_index, test_index in kfold.split(features):
    print(train_index)
    print(test_index)
    print("=" * 80)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [13]:
count = 0
cv_accuracy = []

for train_index, test_index in kfold.split(features):
    count += 1
    
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]
    
    train_targets = pd.DataFrame(y_train)
    test_targets = pd.DataFrame(y_test)
    
    #학습 및 예측
    decision_tree_classifier.fit(X_train, y_train)
    prediction = decision_tree_classifier.predict(X_test)
    
    # 정확도 측정
    accuracy = np.round(accuracy_score(y_test, prediction), 4)
    
    cv_accuracy.append(accuracy)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f"\n# {count} 교차 검증 정확도: {accuracy}, 학습 데이터 크기: {train_size}, 검증 데이터 크기: {test_size}")
    print(f"#{count} 학습 타겟 데이터 분포: \n{train_targets.value_counts()}")
    print(f"#{count} 검증 타겟 데이터 분포: \n{test_targets.value_counts()}")
    print(f"#{count} 학습 세트 인덱스: {train_index}")
    print(f"#{count} 검증 세트 인덱스: {test_index}")
    print("=" * 100)

# 폴드 별 검증 정확도를 합하여 평균 정확도 계산
print(f"▶ 평균 검증 정확도: {np.mean(cv_accuracy)}")


# 1 교차 검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 학습 타겟 데이터 분포: 
1    50
2    50
0    20
Name: count, dtype: int64
#1 검증 타겟 데이터 분포: 
0    30
Name: count, dtype: int64
#1 학습 세트 인덱스: [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
#1 검증 세트 인덱스: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

# 2 교차 검증 정확도: 1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 학습 타겟 데이터 분포: 
2    50
1    40
0    30
Name: count, dtype: int64
#2 검증 타겟 데이터 분포: 
0    20
1    10
Name: count, dtype: int64
#2 학습 세트 인덱스: [  0   1   2   3   4   5   

##### 타겟 데이터의 분포를 동일하게 교차 검증 진행

##### StratifiedFold.split(features, targets)
- features와 targets 모두 전달하고 학습용, 검증용 행 번호를 array로 리턴한다.

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

decision_tree_classifier = DecisionTreeClassifier(min_samples_leaf=6, random_state=124)

# 5개의 폴드 세트로 분리
skfold = StratifiedKFold(n_splits=5)

In [18]:
count = 0
cv_accuracy = []

for train_index, test_index in skfold.split(features, targets):
    count += 1
    
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = targets[train_index], targets[test_index]
    
    train_targets = pd.DataFrame(y_train)
    test_targets = pd.DataFrame(y_test)
    
    #학습 및 예측
    decision_tree_classifier.fit(X_train, y_train)
    prediction = decision_tree_classifier.predict(X_test)
    
    # 정확도 측정
    accuracy = np.round(accuracy_score(y_test, prediction), 4)
    
    cv_accuracy.append(accuracy)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f"\n# {count} 교차 검증 정확도: {accuracy}, 학습 데이터 크기: {train_size}, 검증 데이터 크기: {test_size}")
    print(f"#{count} 학습 타겟 데이터 분포: \n{train_targets.value_counts()}")
    print(f"#{count} 검증 타겟 데이터 분포: \n{test_targets.value_counts()}")
    print(f"#{count} 학습 세트 인덱스: {train_index}")
    print(f"#{count} 검증 세트 인덱스: {test_index}")
    print("=" * 100)

# 폴드 별 검증 정확도를 합하여 평균 정확도 계산
print(f"▶ 평균 검증 정확도: {np.mean(cv_accuracy)}")


# 1 교차 검증 정확도: 0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
#1 검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: count, dtype: int64
#1 학습 세트 인덱스: [ 10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27
  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]

# 2 교차 검증 정확도: 0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 학습 타겟 데이터 분포: 
0    40
1    40
2    40
Name: count, dtype: int64
#2 검증 타겟 데이터 분포: 
0    10
1    10
2    10
Name: c

##### 편하게 수행할 수 있는 교차 검증

##### cross_val_score(estimator, x, y, cv, scoring)
- estimator: classifier 종류 모델이면 내부적으로 stratified K-Fold로 진행된다.
- x: featuers
- y: targets
- scoring: 평가 함수, 정확도(accuracy)외에 다른 것은 다른 장에서 배운다.
- cv: 폴드 세트 개수

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
decision_tree_classifier = DecisionTreeClassifier(random_state=124, min_samples_leaf=6)

features, targets = iris.data, iris.target

score = cross_val_score(decision_tree_classifier, features, targets, scoring='accuracy', cv=5)
print('교차 검증별 정확도: {}'.format(score))
print('평균 정확도: {}'.format(np.mean(score)))

교차 검증별 정확도: [0.96666667 0.96666667 0.9        0.86666667 1.        ]
평균 정확도: 0.9400000000000001


#### GridSearchCV(estimator, param_grid, cv, refit, return_train_score)
- estimator: 학습할 모델 객체 작성
- param_grid: dict형태로 전달해야 하며, 주요 key값은 max_depth, min_samples_split이다.
- cv: 폴드 세트 개수
- refit: 최적의 하이퍼 파라미터로 전달한 모델 객체를 다시 훈련하고자 할 때 True를 전달한다, 디폴트는 True.
- return_train_score: 교차 검증 점수를 가져올 지에 대해 True 또는 False를 전달한다.

In [22]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습 데이터와 테스트 데이터를 분리한다.
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=124)
decision_tree_classifier = DecisionTreeClassifier()

# max_depth: 노드가 생성되는 최대 깊이 수 제한
# min_sample_split: 최소 샘플 개수 제한
parameters = {'max_depth': [2, 3, 4], 'min_samples_split': [6, 7]}

In [23]:
grid_decision_tree_classifier = GridSearchCV(decision_tree_classifier
                                             , param_grid=parameters
                                             , cv=3
                                             , refit=True
                                             , return_train_score=True)

grid_decision_tree_classifier.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4], 'min_samples_split': [6, 7]},
             return_train_score=True)

In [24]:
grid_decision_tree_classifier.cv_results_

{'mean_fit_time': array([0.00033267, 0.00066789, 0.00132505, 0.00066511, 0.00064429,
        0.00099881]),
 'std_fit_time': array([0.00047047, 0.00047228, 0.00047228, 0.00047031, 0.00045564,
        0.00081449]),
 'mean_score_time': array([0.00033172, 0.00099579, 0.00033545, 0.0006636 , 0.00034245,
        0.00065128]),
 'std_score_time': array([4.69122522e-04, 5.28599076e-06, 4.74404927e-04, 4.69235560e-04,
        4.84295387e-04, 4.60818249e-04]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[6, 7, 6, 7, 6, 7],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split': 6},
  {'max_depth': 2, 'min_samples_split': 7},
  {'max_depth': 3, 'min_samples_split': 6},
  {'max_depth': 3, 'min_samples_split': 7},
  {'ma

In [26]:
import pandas as pd

scores_df = pd.DataFrame(grid_decision_tree_classifier.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

params  mean_test_score  rank_test_score  \
0  {'max_depth': 2, 'min_samples_split': 6}         0.975000                3   
1  {'max_depth': 2, 'min_samples_split': 7}         0.966667                6   
2  {'max_depth': 3, 'min_samples_split': 6}         0.975000                3   
3  {'max_depth': 3, 'min_samples_split': 7}         0.991667                1   
4  {'max_depth': 4, 'min_samples_split': 6}         0.975000                3   
5  {'max_depth': 4, 'min_samples_split': 7}         0.991667                1   

   split0_test_score  split1_test_score  split2_test_score  
0              0.975              0.975              0.975  
1              0.950              0.975              0.975  
2              0.950              0.975              1.000  
3              1.000              0.975              1.000  
4              0.950              0.975              1.000  
5              1.000              0.975              1.000

In [28]:
print(f'GridSearchCV 최적 파라미터: {grid_decision_tree_classifier.best_params_}')
print(f'GridSearchCV 최고 정확도: {grid_decision_tree_classifier.best_score_}')

prediction = grid_decision_tree_classifier.predict(X_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, prediction)}')

# refit 된 객체는 best_estimator_로 가져올 수 있으며,
# 이미 grid_decision_tree_classifier객체를 GridSearchCV로 작업하여 생성했기 때문에
# 결과는 똑같이 나온다.
estimator = grid_decision_tree_classifier.best_estimator_
prediction = estimator.predict(X_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, prediction)}')

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 7}
GridSearchCV 최고 정확도: 0.9916666666666667
테스트 데이터 세트 정확도: 0.9
테스트 데이터 세트 정확도: 0.9
